In [19]:
import numpy as np
from scipy.optimize import minimize
np.set_printoptions(threshold='nan')

## prepare data set
- this bloc is for preparing the data matrix has_rating[i,j] 1 if a paticular user has rated a particular movie, 0. otherwise rating[i,j] real rating that given by particular users **value exist if and only if coresponding has_rating[i,j]==1
- number of users can be optain by len(names) and number of movies can be optain by len(movies)

In [20]:
import StringIO
import csv
with open('resource - Sheet1.csv') as f:
    s = StringIO.StringIO(f.read()) 
    reader = csv.reader(s, delimiter=',')
    raw = [row for row in reader]

def get_names(raw):
    return raw[2][2:]

def get_movies(raw):
    movies = []
    for i in range(3, len(raw)):
        movies.append(raw[i][1])
    return movies

def get_rating(raw):
    
    names = get_names(raw)
    movies = get_movies(raw)
    name_offset = 2
    movies_offset = 3
    
    n_names = len(names)
    n_movies = len(movies)

    has_rating = np.zeros((n_names, n_movies))
    rating = np.zeros((n_names, n_movies))
    
    for ir in range(n_movies):
        
        this_row = raw[movies_offset + ir]

        for ip in range(n_names):
            
            this_cell =  this_row[name_offset + ip]
            hr = this_cell.isdigit()
            has_rating[ip, ir] = 1 if hr else 0
            if hr:
                rating[ip, ir] = int(this_cell)
                
    return names, movies, has_rating, rating

names, movies, has_rating, rating = get_rating(raw)

print "name", names
print "movies", movies
print "has_rating", has_rating.T
print "rating", rating.T



name ['Alice', 'Bob', 'Carol', 'Dave', 'Eve']
movies ['love', 'Romance', 'Cute', 'Nonstop', 'sword', 'iron man']
has_rating [[ 1.  1.  1.  1.  0.]
 [ 1.  0.  0.  1.  0.]
 [ 0.  1.  1.  0.  0.]
 [ 1.  1.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.]]
rating [[ 5.  5.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.]
 [ 0.  0.  5.  4.  0.]
 [ 0.  0.  5.  0.  0.]
 [ 0.  0.  0.  0.  1.]]


big_theta is when we assume the users have given thier preferences to us 

In [21]:
big_theta = np.array([[5.,0],[5.,0],[0.,5],[0.,5]])
print big_theta

[[ 5.  0.]
 [ 5.  0.]
 [ 0.  5.]
 [ 0.  5.]]


## Implementation of Detail Mean Normalization
- the seection below here is implemented to handle the case of newly added user that has never rated any movie before 
- notice that in our data set we have Eve who never rate any movie so we can think of her as newly added user

In [22]:
# init the average matrix
average = np.zeros((len(movies),1))
print "size",has_rating.T.shape
# find average value for each row 
print "len name",len(names)
print "len movies",len(movies)
for i in range (0,len(movies)):
    print "i", i 
    sum_each_row = 0
    count_num = 0
    for j in range (0,len(names)):
        print "j", j
        if has_rating.T[i][j] == 1:
            sum_each_row = sum_each_row + rating.T[i][j]
            count_num = count_num+1
    print count_num
    average[i][0] = sum_each_row/count_num
    
print average

size (6, 5)
len name 5
len movies 6
i 0
j 0
j 1
j 2
j 3
j 4
4
i 1
j 0
j 1
j 2
j 3
j 4
2
i 2
j 0
j 1
j 2
j 3
j 4
2
i 3
j 0
j 1
j 2
j 3
j 4
4
i 4
j 0
j 1
j 2
j 3
j 4
3
i 5
j 0
j 1
j 2
j 3
j 4
1
[[ 2.5       ]
 [ 2.5       ]
 [ 2.        ]
 [ 2.25      ]
 [ 1.66666667]
 [ 1.        ]]


# running data after Implementation of Detail Mean Normalization
- on the section below here we re-run our main again with the new rating matrix  

In [23]:
print "original rating",rating.T
# re-init value for rating matrix
for i in range (0,len(movies)):
    for j in range (0,len(names)):
        if has_rating.T[i][j] == 1:
            rating.T[i][j] = rating.T[i][j] - average[i]
print "rating after DMN",rating.T 

original rating [[ 5.  5.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.]
 [ 0.  0.  5.  4.  0.]
 [ 0.  0.  5.  0.  0.]
 [ 0.  0.  0.  0.  1.]]
rating after DMN [[ 2.5         2.5        -2.5        -2.5         0.        ]
 [ 2.5         0.          0.         -2.5         0.        ]
 [ 0.          2.         -2.          0.          0.        ]
 [-2.25       -2.25        2.75        1.75        0.        ]
 [-1.66666667 -1.66666667  3.33333333  0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


## Getting the best weight
- start Linear Regression class below here 

In [24]:
class LinearRegression: 
    
    # init all the data so it can easily accessed by all methods
    def __init__(self,has_rating,real_rating,fea,lamda,names,movies):
        #has_rating and real_rating here are already transposed
        self.has_rating = has_rating
        self.fea=fea
        self.real_rating = real_rating
        self.lamda=lamda
        self.names = names
        self.movies = movies
        self.num_names = len(names)
        self.num_movies = len(movies) 
        
    
    # cost function use for minimizing  
    def cost_finction(self,w):
        
        # 1/2 constant
        con=float(1)/2
        w = w.reshape(self.num_names+self.num_movies,self.fea)
                
        # set of all guessing theta values 
        theta = w[0:self.num_names][0:]
       
        # set of all guessing x values
        x = w[self.num_names:self.num_names+self.num_movies][0:]
        
        # 3 summation variables in the equation   
        sum_linear = 0
        sum_theta = 0
        sum_x = 0

        # below here we will sum over all pairs (i,j) for which has_rating(i,j) == 1 
        for i in range(0,self.num_movies):
            for j in range(0,self.num_names):
                if self.has_rating[i][j] == 1:
                    sum_linear = sum_linear + ((np.dot(theta[j].T,x[i]))-self.real_rating[i,j])**2 
                    
        # below here we will sum over the nomalization terms for x(k)
        for i in range(0,self.num_movies):
            for k in range (0,self.fea):
                sum_x = sum_x + (w[i][k]**2)
        
        # below here we will sum over the nomalization terms for theta(k)
        for i in range(0,self.num_names):
            for k in range (0,self.fea):
                sum_theta = sum_theta + (theta[i][k]**2)  
        
        return con*sum_linear+((float(self.lamda)/2)*sum_x)+((float(self.lamda)/2)*sum_theta)

# these two variable can be change 
number_feature = 2
lamda = 0.01

linear_obj = LinearRegression(has_rating.T,rating.T,number_feature,lamda,names,movies)

# initialize random value range 0-5 (user preference rate, theta) to a two-d array 
guess_theta = np.random.uniform(0.1,1,[len(names),number_feature])
#print guess_theta

# initialize random value range 0-1 (movie characteristic rate) to a two-d array 
guess_x = np.random.uniform(0.1,1,[len(movies),number_feature])
#print guess_x

# concatinate guess_theta and guess_x so it can be pass together to the minimize function
weight_value = np.concatenate((guess_theta,guess_x),axis=0)
#print weight_value

# looking for the weight that will  give the dot product closest to the real value
minimize_weight = minimize(linear_obj.cost_finction,weight_value)
result_w = minimize_weight.x.reshape(len(names)+len(movies),2)

# separate final theta and x from the final w
result_theta = result_w[0:len(names)][0:]
print result_theta
result_x = result_w[len(names):len(names)+len(movies)][0:]
print result_x

# setting up output array
out = np.zeros((len(movies),len(names)))
for i in range(0,len(movies)):
            for j in range(0,len(names)):
                    out[i][j] = int(np.dot(result_theta[j].T,result_x[i]))+int(average[i])
print "output",out

[[-0.14926402  0.91407401]
 [-0.14904215  0.912724  ]
 [ 0.16456703 -1.01100707]
 [ 0.10035365 -0.95699568]
 [-0.00187237  0.0028306 ]]
[[ -4.03939605e-01   2.56333282e+00]
 [ -2.16533180e+00   2.38319374e+00]
 [ -4.44650771e+02  -7.04077268e+01]
 [  1.25539593e+01  -5.13223416e-01]
 [ -2.82746990e+03  -4.63536460e+02]
 [ -1.35929226e+02  -8.99609846e+01]]
output [[   4.    4.    0.    0.    2.]
 [   4.    4.    0.    0.    2.]
 [   4.    4.    1.   24.    2.]
 [   0.    0.    4.    3.    2.]
 [   0.    0.    4.  160.    4.]
 [ -60.  -60.   69.   73.    1.]]
